# What's the weather like as we approach the equator?"

# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
Weather_Data = pd.read_csv("output_data/Weather_Data.csv")
df=pd.DataFrame(Weather_Data)
df

,City,Latitude,Longitude,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.12,-134.97,68.702,60,0,4.56,PF,1595207783
1,Busselton,-33.65,115.33,46.004,92,0,4.13,AU,1595207783
2,Arman',59.70,150.17,62.816,55,4,3.16,RU,1595207506
3,Mataura,-46.19,168.86,47.372,67,100,1.60,NZ,1595207783
4,Saskylakh,71.92,114.08,60.584,58,33,4.92,RU,1595207751
...,...,...,...,...,...,...,...,...,...
541,Kisangani,0.52,25.20,72.140,73,94,0.64,CD,1595207830
542,Constantine,36.37,6.61,66.200,77,0,2.10,DZ,1595207831
543,Adré,13.47,22.20,73.886,79,100,5.39,TD,1595207831
544,Aswān,24.09,32.91,86.270,19,71,4.60,EG,1595207831


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

locations = df[["Latitude", "Longitude"]].astype(float)
weights = df["Humidity"].astype(int)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Create DF for Weather Conditions (max temp 70<x<80, wind < 10, cloudiness = 0, Drop cities w/o conditions)
ideal_df = df.loc[(df["Max Temp (F)"]>70) & (df["Max Temp (F)"]<80) & (df["Wind Speed"]<10) & (df["Cloudiness"] == 0)]
ideal_df = ideal_df.reset_index()
idf=ideal_df.drop(columns = "index")
idf.head() #USE IDF FOR DEFAULT DF

#idf #for all rows

#No null values to drop UNCHECK THESE TO DROP ROWS
#dropped_df = idf.dropna()
#dropped_df
#dropped_df.head()

,City,Latitude,Longitude,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Kerch,45.36,36.48,77.252,69,0,4.78,UA,1595207786
1,Calvià,39.57,2.51,71.996,68,0,1.50,ES,1595207789
2,Srednekolymsk,67.45,153.68,76.622,35,0,5.49,RU,1595207789
3,Guerrero Negro,27.98,-114.06,73.490,72,0,7.77,MX,1595207542
4,Fonte Boa,-2.50,-66.27,74.210,93,0,0.68,BR,1595207792


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=idf #[["City", "Country","Latitude", "Longitude", "Max Temp (F)", "Humidity", "Cloudiness", "Wind Speed", "Date"]].copy #or change idf to dropped_df in case of Null values, check above

#Add Hotel Name as a column
hotel_df["Hotel Name"] = " "
hotel_df.head()

,City,Latitude,Longitude,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kerch,45.36,36.48,77.252,69,0,4.78,UA,1595207786,
1,Calvià,39.57,2.51,71.996,68,0,1.50,ES,1595207789,
2,Srednekolymsk,67.45,153.68,76.622,35,0,5.49,RU,1595207789,
3,Guerrero Negro,27.98,-114.06,73.490,72,0,7.77,MX,1595207542,
4,Fonte Boa,-2.50,-66.27,74.210,93,0,0.68,BR,1595207792,


In [7]:
#Search Parameters (5000m)
#Base URL: https://developers.google.com/places/web-service/search#PlaceSearchRequests
#Type: https://developers.google.com/places/web-service/supported_types

hotel_name = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": "5000",
    "type": "lodging",
    "key": g_key,
}

#Loop through IDF to append Lat/Lng to params for query request

for index, row in hotel_df.iterrows():    
    #For each row in hotel_df, get the Lat/Lng & Pass it into params for search 
    #MAKE SURE LAT AND LNG ARE NOT IN PARARENTHESIS
    location = row[["Latitude","Longitude"]]
    #pass held lat and lng for each iteration
    params['location'] = f"{row['Latitude']}, {row['Longitude']}"
    
    #Request Json
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_name.append(response['results'][0]['name'])
    except:
        hotel_name.append(np.NaN)

In [8]:
#print(hotel_name)
hotel_df["Hotel Name"] = hotel_name
hotel_df

,City,Latitude,Longitude,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kerch,45.36,36.48,77.252,69,0,4.78,UA,1595207786,Khostel
1,Calvià,39.57,2.51,71.996,68,0,1.50,ES,1595207789,Castell Son Claret
2,Srednekolymsk,67.45,153.68,76.622,35,0,5.49,RU,1595207789,NaN
3,Guerrero Negro,27.98,-114.06,73.490,72,0,7.77,MX,1595207542,"Casa Laguna, Bed & Breakfast"
4,Fonte Boa,-2.50,-66.27,74.210,93,0,0.68,BR,1595207792,NaN
5,Cherskiy,68.75,161.30,73.238,38,0,2.00,RU,1595207794,Gostinitsa
6,Villamontes,-21.25,-63.50,73.166,63,0,0.76,BO,1595207797,Hotel Raldes
7,Labuhan,-6.88,112.21,78.962,78,0,3.37,ID,1595207799,bador sepret
8,Kamari,36.80,25.82,73.004,71,0,0.89,GR,1595207801,Minoa Hotel Amorgos
9,Ekibastuz,51.67,75.37,72.860,58,0,2.38,KZ,1595207803,NaN


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
gmaps.configure(api_key = g_key)

locations = hotel_df[["Latitude", "Longitude"]].astype(float)
weights = hotel_df["Humidity"].astype(int)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))